## Dependencies

In [1]:
import glob
import warnings
from tensorflow_hub import KerasLayer
from tensorflow.keras import Model
from tensorflow.keras.layers import Input, Dense, Dropout, GlobalAveragePooling1D, SpatialDropout1D, Concatenate
from googleqa_utilityscript import *
from googleqa_map_utilityscript import *
import bert_tokenization as tokenization
from transformers import BertConfig, BertTokenizer, TFBertModel


SEED = 0
seed_everything(SEED)
warnings.filterwarnings("ignore")

## Load data

In [2]:
BERT_PATH = '/kaggle/input/bert-base-uncased-huggingface-transformer/bert-base-uncased-tf_model.h5'
VOCAB_PATH = '/kaggle/input/bert-base-uncased-huggingface-transformer/bert-base-uncased-vocab.txt'
model_path_list = glob.glob('/kaggle/input/141-googleq-a-train-3fold-bert-base-unc-regression/' + '*last_epoch.h5')
model_path_list.sort()
print('Models to predict:')
print(*model_path_list, sep = "\n")

test = pd.read_csv('/kaggle/input/google-quest-challenge/test.csv')

print('Test samples: %s' % len(test))
display(test.head())

Models to predict:
/kaggle/input/141-googleq-a-train-3fold-bert-base-unc-regression/model_fold_1_last_epoch.h5
/kaggle/input/141-googleq-a-train-3fold-bert-base-unc-regression/model_fold_2_last_epoch.h5
/kaggle/input/141-googleq-a-train-3fold-bert-base-unc-regression/model_fold_3_last_epoch.h5
Test samples: 476


,qa_id,question_title,question_body,question_user_name,question_user_page,answer,answer_user_name,answer_user_page,url,category,host
0,39,Will leaving corpses lying around upset my pri...,I see questions/information online about how t...,Dylan,https://gaming.stackexchange.com/users/64471,There is no consequence for leaving corpses an...,Nelson868,https://gaming.stackexchange.com/users/97324,http://gaming.stackexchange.com/questions/1979...,CULTURE,gaming.stackexchange.com
1,46,Url link to feature image in the portfolio,I am new to Wordpress. i have issue with Featu...,Anu,https://wordpress.stackexchange.com/users/72927,I think it is possible with custom fields.\n\n...,Irina,https://wordpress.stackexchange.com/users/27233,http://wordpress.stackexchange.com/questions/1...,TECHNOLOGY,wordpress.stackexchange.com
2,70,"Is accuracy, recoil or bullet spread affected ...","To experiment I started a bot game, toggled in...",Konsta,https://gaming.stackexchange.com/users/37545,You do not have armour in the screenshots. Thi...,Damon Smithies,https://gaming.stackexchange.com/users/70641,http://gaming.stackexchange.com/questions/2154...,CULTURE,gaming.stackexchange.com
3,132,Suddenly got an I/O error from my external HDD,I have used my Raspberry Pi as a torrent-serve...,robbannn,https://raspberrypi.stackexchange.com/users/17341,Your Western Digital hard drive is disappearin...,HeatfanJohn,https://raspberrypi.stackexchange.com/users/1311,http://raspberrypi.stackexchange.com/questions...,TECHNOLOGY,raspberrypi.stackexchange.com
4,200,Passenger Name - Flight Booking Passenger only...,I have bought Delhi-London return flights for ...,Amit,https://travel.stackexchange.com/users/29089,I called two persons who work for Saudia (tick...,Nean Der Thal,https://travel.stackexchange.com/users/10051,http://travel.stackexchange.com/questions/4704...,CULTURE,travel.stackexchange.com


In [3]:
question_target_cols = ['question_asker_intent_understanding','question_body_critical', 'question_conversational', 
                        'question_expect_short_answer', 'question_fact_seeking', 'question_has_commonly_accepted_answer',
                        'question_interestingness_others', 'question_interestingness_self', 'question_multi_intent', 
                        'question_not_really_a_question', 'question_opinion_seeking', 'question_type_choice',
                        'question_type_compare', 'question_type_consequence', 'question_type_definition', 
                        'question_type_entity', 'question_type_instructions', 'question_type_procedure',
                        'question_type_reason_explanation', 'question_type_spelling', 'question_well_written']
answer_target_cols = ['answer_helpful', 'answer_level_of_information', 'answer_plausible', 'answer_relevance',
                      'answer_satisfaction', 'answer_type_instructions', 'answer_type_procedure', 
                      'answer_type_reason_explanation', 'answer_well_written']
target_cols = question_target_cols + answer_target_cols

## Pre-process data

In [4]:
text_features = ['question_title', 'question_body', 'answer']

# for feature in text_features:
#     # Lower
#     test[feature] = test[feature].apply(lambda x: x.lower())
#     # Map misspellings
#     test[feature] = test[feature].apply(lambda x: map_misspellings(x))
#     # Map contractions
#     test[feature] = test[feature].apply(lambda x: map_contraction(x))
#     # Trim text
#     test[feature] = test[feature].apply(lambda x: x.strip())

# Model parameters

In [5]:
N_CLASS = len(target_cols)
MAX_SEQUENCE_LENGTH = 512

## Test set

In [6]:
tokenizer = BertTokenizer.from_pretrained(VOCAB_PATH, do_lower_case=True)

# Test features
X_test = compute_input_arays(test, text_features, tokenizer, MAX_SEQUENCE_LENGTH)

# Model

In [7]:
bert_config = BertConfig()
bert_config.output_hidden_states=False

def model_fn():
    input_word_ids = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='input_word_ids')
    input_masks = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='input_masks')
    segment_ids = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='segment_ids')

    bert_model = TFBertModel.from_pretrained(BERT_PATH, config=bert_config)
    sequence_output, pooled_output = bert_model([input_word_ids, input_masks, segment_ids])

    x = GlobalAveragePooling1D()(sequence_output)
    x = Dropout(0.2)(x)
    output = Dense(N_CLASS, activation="linear", name="output")(x)

    model = Model(inputs=[input_word_ids, input_masks, segment_ids], outputs=output)
    
    return model

# Make predictions

In [8]:
Y_test = np.zeros((len(test), N_CLASS))

for model_path in model_path_list:
    model = model_fn()
    model.load_weights(model_path)
    Y_test += model.predict(X_test) / len(model_path_list)

In [9]:
submission = pd.read_csv('/kaggle/input/google-quest-challenge/sample_submission.csv')
submission[target_cols] = Y_test
submission.to_csv("submission.csv", index=False)

submission = submission.clip(0, 1)

display(submission.head())
display(submission.describe())

,qa_id,question_asker_intent_understanding,question_body_critical,question_conversational,question_expect_short_answer,question_fact_seeking,question_has_commonly_accepted_answer,question_interestingness_others,question_interestingness_self,question_multi_intent,...,question_well_written,answer_helpful,answer_level_of_information,answer_plausible,answer_relevance,answer_satisfaction,answer_type_instructions,answer_type_procedure,answer_type_reason_explanation,answer_well_written
0,1,0.927099,0.658920,0.221862,0.447656,0.571087,0.491424,0.666752,0.631167,0.646074,...,0.938334,0.882922,0.536263,0.947842,0.932386,0.802899,0.080055,0.000000,0.867903,0.963462
1,1,0.892154,0.477231,0.000540,0.801773,0.721849,0.861059,0.502440,0.411132,0.128659,...,0.692702,0.956327,0.617152,0.966104,0.945474,0.875112,0.882755,0.118832,0.158680,0.913463
2,1,0.913571,0.659139,0.028106,0.774391,0.807588,0.892691,0.601081,0.500096,0.152602,...,0.906713,0.928747,0.604462,0.964490,0.943482,0.875925,0.058575,0.067307,0.824822,0.934951
3,1,0.856083,0.405730,0.011364,0.712281,0.701690,0.848886,0.487464,0.406339,0.130589,...,0.672797,0.976612,0.665705,0.986632,0.995698,0.944175,0.813275,0.185804,0.565130,0.924850
4,1,0.951407,0.438766,0.083760,0.853891,0.699462,0.733233,0.608781,0.584214,0.177148,...,0.733582,0.922080,0.633682,0.967221,0.911436,0.866538,0.227474,0.148513,0.707510,0.944478


,qa_id,question_asker_intent_understanding,question_body_critical,question_conversational,question_expect_short_answer,question_fact_seeking,question_has_commonly_accepted_answer,question_interestingness_others,question_interestingness_self,question_multi_intent,...,question_well_written,answer_helpful,answer_level_of_information,answer_plausible,answer_relevance,answer_satisfaction,answer_type_instructions,answer_type_procedure,answer_type_reason_explanation,answer_well_written
count,476.0,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,...,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000
mean,1.0,0.892544,0.575581,0.043839,0.716978,0.747801,0.770999,0.541726,0.466291,0.260252,...,0.812145,0.928224,0.637825,0.944680,0.951571,0.862140,0.496649,0.138150,0.527534,0.920956
std,0.0,0.048714,0.125810,0.072662,0.107922,0.106403,0.143933,0.054179,0.087166,0.206110,...,0.088817,0.033418,0.049929,0.023377,0.025465,0.041728,0.315156,0.061501,0.258911,0.025621
min,1.0,0.756854,0.273069,0.000000,0.319846,0.262774,0.083365,0.438184,0.277702,0.000000,...,0.611698,0.801220,0.515773,0.872169,0.869245,0.717816,0.000000,0.000000,0.000000,0.853457
25%,1.0,0.857845,0.474086,0.000000,0.657437,0.695817,0.711061,0.501576,0.404931,0.097421,...,0.736256,0.905289,0.602896,0.929223,0.936369,0.834788,0.192327,0.103882,0.322687,0.904763
50%,1.0,0.894485,0.568575,0.014158,0.723530,0.750858,0.815259,0.528061,0.442050,0.208061,...,0.816278,0.929877,0.634929,0.945193,0.952730,0.864155,0.551848,0.141905,0.521112,0.920234
75%,1.0,0.927815,0.676326,0.049594,0.788387,0.813614,0.867796,0.580266,0.503748,0.375428,...,0.885899,0.950671,0.672000,0.961254,0.970282,0.890303,0.781138,0.180441,0.724847,0.937221
max,1.0,1.000000,0.897264,0.503113,1.000000,1.000000,1.000000,0.689321,0.771957,1.000000,...,1.000000,1.000000,0.796291,1.000000,1.000000,0.983753,1.000000,0.300347,1.000000,1.000000
